# Problema 1 -- Grande Polônia
Foram utilizados o **Algoritmo de Dijkstra** e a estrutura de **Fila de Prioridades Min-Heap** para solucionar o problema. Para cada $\alpha_i$, calculou-se o seu caminho para cada $\beta_i$ utilizando o **Algoritmo de Caminho Mínimo de Dijkstra**. Foram colocados todas as informações em um **Min-Heap**, o qual utilizou como chave **peso total da viagem**. Veja como foi realizado em Python.

In [66]:
import networkx as nx
import heapq

# Estrutura a ser usada no Heap
class Item:
    def __init__(self, peso_total: int, where: int, to: int, path: list):
        self.peso_total = peso_total    # Peso total do caminho
        self.where = where              # De qual alfa sai
        self.to = to                    # Para qual beta vai
        self.path = path                # Caminho a seguir
    def __lt__(self, other):
        return self.peso_total < other.peso_total
    def __repr__(self):
        return f"Peso Total: {self.peso_total} - Alfa {self.where + 1} to Beta {self.to - 4}"

In [67]:
# Enumeração Primitiva
alfa_1     = 0
alfa_2     = 1
alfa_3     = 2
intermed_1 = 3
intermed_2 = 4
beta_1     = 5
beta_2     = 6
beta_3     = 7

In [68]:
# Criando nosso grafo ponderado
G = nx.Graph()
G.add_edges_from(
    [
        (alfa_1, intermed_1,     {"weight": 1}),
        (alfa_1, intermed_2,     {"weight": 3}),
        (alfa_2, intermed_1,     {"weight": 2}),
        (alfa_2, intermed_2,     {"weight": 3}),
        (alfa_3, intermed_2,     {"weight": 1}),
        (alfa_3, beta_3,         {"weight": 4}),
        (intermed_1, beta_1,     {"weight": 2}),
        (intermed_1, beta_2,     {"weight": 4}),
        (intermed_1, intermed_2, {"weight": 1}),
        (intermed_2, beta_1,     {"weight": 3}),
        (intermed_2, beta_2,     {"weight": 5}),
        (intermed_2, beta_3,     {"weight": 2}),
    ]
)

In [69]:
# Dijkstra para cada fonte Alfa
path_alphas = [
    nx.single_source_dijkstra(G, alfa_1),
    nx.single_source_dijkstra(G, alfa_2),
    nx.single_source_dijkstra(G, alfa_3)
]

In [70]:
# Encontrando os caminhos mínimos entre cada alfa e beta
# NÃO PODEMOS DESCARTAR NENHUM CAMINHO!
# Isso porque o menor caminho para alfa_2 é até beta_1, o que tornaria beta_2
# desprotegido
alphas = [alfa_1, alfa_2, alfa_3]
betas = [beta_1, beta_2, beta_3]
todos_caminhos = []
for alpha in alphas:
    for beta in betas:
        todos_caminhos.append(
            Item(path_alphas[alpha][0][beta], alpha, beta, path_alphas[alpha][1][beta])
        )
heapq.heapify( todos_caminhos )

# Próximo Passo
Com as rotas definidas, precisamos manejar as tropas adequadamente. $\alpha_i$ não pode ter menos que $0$ tropas, assim como $\beta_i$ tem um limite especificado. Estes são os limites:
$$\beta_1 \rightarrow 2 \textit{ divisões} $$
$$\beta_2 \rightarrow 1 \textit{ divisão} $$
$$\beta_3 \rightarrow 1 \textit{ divisão}$$

A partir disso, vamos usar arrays para verificar o *status* de cada vértice (quantas tropas estão lá) e o *limite* de cada vértices.

In [71]:
status  = [ 2,  2,  1,  -1, -1, 0, 0, 0]   # -1 são inválidos - são os vértices intermediários
limites = [-1, -1, -1 , -1, -1, 2, 1, 1]   # -1 são inválidos - são os vértices intermediários e alfas

In [72]:
# Enquanto nosso heap não estiver vazio
caminhos_percorridos_e_tropas = []   # Armazena as decisões tomadas pelos generais!
while len(todos_caminhos) > 0:
    popped_path = heapq.heappop(todos_caminhos)  # Retira o menor do Heap
    alpha = popped_path.where
    beta  = popped_path.to
    max_envio = limites[beta] - status[beta]     # Quanto dá pra enviar para beta
    
    print("==================================")
    print(f"Alpha {alpha + 1} -> Beta {beta - 4}")
    print(f"Status[Alpha {alpha + 1}] = {status[alpha]}")
    print(f"Status[Beta {beta - 4}] = {status[beta]}\t Limite {limites[beta]}")
    print(f"Máximo que dá pra enviar: {max_envio} ")
    input()

    # Se ainda tiver tropas em alpha e dá pra enviar, envia o máximo que dá
    if status[alpha] > 0 and max_envio > 0:
        # Dois casos
        if max_envio > status[alpha]:
            # Envia tudo em alpha
            print(f"Envia tudo!")
            print(f"Alpha {alpha + 1} envia {status[alpha]}")
            tropas_enviadas = status[alpha]
            status[beta] += status[alpha]
            status[alpha] = 0
        else:
            # Envia o máximo que beta aceita
            print(f"Envia o máximo que beta aceita!")
            print(f"Alpha {alpha + 1} envia {max_envio} para Beta {beta - 4}")
            tropas_enviadas = max_envio
            status[beta] += max_envio
            status[alpha] -= max_envio
        caminhos_percorridos_e_tropas.append(
            (popped_path.path, tropas_enviadas, popped_path.peso_total)
        )
print("==================================")
print("Caminhos percorridos")
print(caminhos_percorridos_e_tropas)

Alpha 3 -> Beta 3
Status[Alpha 3] = 1
Status[Beta 3] = 0	 Limite 1
Máximo que dá pra enviar: 1 


Envia o máximo que beta aceita!
Alpha 3 envia 1 para Beta 3
Alpha 1 -> Beta 1
Status[Alpha 1] = 2
Status[Beta 1] = 0	 Limite 2
Máximo que dá pra enviar: 2 


Envia o máximo que beta aceita!
Alpha 1 envia 2 para Beta 1
Alpha 3 -> Beta 1
Status[Alpha 3] = 0
Status[Beta 1] = 2	 Limite 2
Máximo que dá pra enviar: 0 


Alpha 1 -> Beta 3
Status[Alpha 1] = 0
Status[Beta 3] = 1	 Limite 1
Máximo que dá pra enviar: 0 


Alpha 2 -> Beta 1
Status[Alpha 2] = 2
Status[Beta 1] = 2	 Limite 2
Máximo que dá pra enviar: 0 


Alpha 2 -> Beta 3
Status[Alpha 2] = 2
Status[Beta 3] = 1	 Limite 1
Máximo que dá pra enviar: 0 


Alpha 1 -> Beta 2
Status[Alpha 1] = 0
Status[Beta 2] = 0	 Limite 1
Máximo que dá pra enviar: 1 


Alpha 3 -> Beta 2
Status[Alpha 3] = 0
Status[Beta 2] = 0	 Limite 1
Máximo que dá pra enviar: 1 


Alpha 2 -> Beta 2
Status[Alpha 2] = 2
Status[Beta 2] = 0	 Limite 1
Máximo que dá pra enviar: 1 


Envia o máximo que beta aceita!
Alpha 2 envia 1 para Beta 2
Caminhos percorridos
[([2, 4, 7], 1, 3), ([0, 3, 5], 2, 3), ([1, 3, 6], 1, 6)]


# A partir disso, temos os caminhos!
Indicado pela lista acima que contém tuplas com a seguinte estrutura:
( caminho: list, tropas_enviadas: int, peso_total: int)

In [60]:
print(caminhos_percorridos_e_tropas)

[([2, 4, 7], 1, 3), ([0, 3, 5], 2, 3), ([1, 3, 6], 1, 6)]


In [61]:
for i in caminhos_percorridos_e_tropas:
    print(f"Alfa {i[0][0] + 1} -> Beta {i[0][-1] - 4}\tTropas Enviadas = {i[1]}\tPeso Total = {i[2]}")

Alfa 3 -> Beta 3	Tropas Enviadas = 1	Peso Total = 3
Alfa 1 -> Beta 1	Tropas Enviadas = 2	Peso Total = 3
Alfa 2 -> Beta 2	Tropas Enviadas = 1	Peso Total = 6


In [62]:
missao_peso_total = 0
for i in caminhos_percorridos_e_tropas:
    missao_peso_total += i[2]
print(f"Peso total da missão = {missao_peso_total}")

Peso total da missão = 12


In [63]:
# Todos os betas estão satisfeitos?
for i in [beta_1, beta_2, beta_3]:
    print(f"Beta {i - 4}: Status = {status[i]}\tLimite = {limites[i]}\tSatisfeito = {status[i] == limites[i]}")

Beta 1: Status = 2	Limite = 2	Satisfeito = True
Beta 2: Status = 1	Limite = 1	Satisfeito = True
Beta 3: Status = 1	Limite = 1	Satisfeito = True


In [64]:
# E quanto a alfa - Onde ficou o Regimento extra?
regimento_extra = -1
for i in [alfa_1, alfa_2, alfa_3]:
    if status[i] > 0:
        regimento_extra = i
if regimento_extra == -1:
    print("Erro! Não existe regimentos extras!")
else:
    print(f"O regimento extra está em Alfa {regimento_extra + 1}")

O regimento extra está em Alfa 2
